## Testing LLM Factory

In [ ]:
from src.common.llm import LLM
from src.common.types import LLMConfig

llm_factory = LLM(provider="openai", config={"model": "gpt-4o-mini", "reasoning": "thinking"})

llm = llm_factory.get_llm()
llm.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm just a digital assistant so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 13, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bv2sCwaCsBJWcUG1dYbRUedmJrMes', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e52e57ab-190f-417d-9989-46416008e517-0', usage_metadata={'input_tokens': 13, 'output_tokens': 36, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from src.agent.graph import build_graph
import time

@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    time.sleep(10)
    return a + b

@tool
def sub(a: int, b: int) -> int:
    """Subtract two numbers"""
    time.sleep(10)
    return a - b

@tool
def mul(a: int, b: int) -> int:
    """Multiply two numbers"""  
    time.sleep(10)
    return a * b

tools = [add, sub, mul]

graph = build_graph(tools)

response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="What is 2 + 2?")
        ],
        "llm_config": {
            "model": "gpt-4o-mini",
        },
        "llm_provider": "openai",
        "thread_id": "123",
    }
)

for message in response["messages"]:
    print(message.type)
    print(message)
    print("-"*100)

human
content='What is 2 + 2?' additional_kwargs={} response_metadata={} id='ece3eae8-cea3-4b68-a468-6fff25d01f0e'
----------------------------------------------------------------------------------------------------
ai
content='' additional_kwargs={'tool_calls': [{'id': 'call_1qn4O4pzJqsSYcKZ7IhJlxcG', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 146, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvT7zuIpMIyEB7J7Kt4ld9rXqE9sO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ebecffbb-3b97-4374-be0e-71bcd70c3fee-0' tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}

In [ ]:
from langgraph import graph
from src.agent.graph import build_graph

graph = build_graph(tools)

for type, chunk in graph.stream(
        {
            "messages": [
                HumanMessage(content="What is 2 + 2?")
            ],
            "llm_config": {
                "model": "gpt-4o-mini",
            },
            "llm_provider": "openai"
        },
        stream_mode=["custom", "messages"]
    ):

    print(type)
    print(chunk)
    print("-"*100)

messages
(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Hgx5nALRMViTE5JbNWlR0LJl', 'function': {'arguments': '', 'name': 'add'}, 'type': 'function'}]}, response_metadata={}, id='run--fa30eeb8-3eab-49c8-ad15-bbc94c700a85', tool_calls=[{'name': 'add', 'args': {}, 'id': 'call_Hgx5nALRMViTE5JbNWlR0LJl', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add', 'args': '', 'id': 'call_Hgx5nALRMViTE5JbNWlR0LJl', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'chatbot', 'langgraph_triggers': ('branch:to:chatbot',), 'langgraph_path': ('__pregel_pull', 'chatbot'), 'langgraph_checkpoint_ns': 'chatbot:33751be2-c983-3faf-da5c-e73c5b85e882', 'checkpoint_ns': 'chatbot:33751be2-c983-3faf-da5c-e73c5b85e882', 'ls_provider': 'openai', 'ls_model_name': 'gpt-3.5-turbo', 'ls_model_type': 'chat', 'ls_temperature': None})
----------------------------------------------------------------------------------------------------
messages
(AIMe